In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick201904final.xlsx")
data = alldata.groupby(['LocationID']).mean()
data1 = data[['Freq','pop_density','commercial_density','road_density','X','Y']]

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density']])
data1[['pop_density','commercial_density','road_density']] = scaler.transform(data1[['pop_density','commercial_density','road_density']])

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
E:\anaconda\lib\site-packages\pandas\core\frame.py:2969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
E:\anaconda\lib\site-packages\pandas\core\frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [30]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])


y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GWRparameter

sel = search_GWRparameter(coords, y, x, kernel = 'bisquare')

bw = sel.search(verbose = True)

bw: 128.0 , score: 3567.95
bw: 97.0 , score: 3521.97
bw: 79.0 , score: 3483.71
bw: 67.0 , score: 3450.59
bw: 60.0 , score: 3430.4
bw: 55.0 , score: 3420.6
bw: 53.0 , score: 3415.45
bw: 51.0 , score: 3412.02
bw: 50.0 , score: 3409.92
bw: 49.0 , score: 3409.35
bw: 49.0 , score: 3409.35


In [3]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GWR

results1 = GWR(coords, y, x,49, kernel='bisquare').fit()

results1.R2

0.9089215955288577

In [32]:
coords = results1.coords
coords = pd.DataFrame(coords)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([betas,tvalues,coords], axis = 1)
final_result.columns=['beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

In [33]:
data2 = pd.merge(data1, final_result, left_on=['X','Y'], right_on=['X','Y'] , how='left')
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)
data2.to_csv(r'E:\\Capstone Project\\Data\\paper\\minor\\pick201904beta.csv',index=True,sep=',')

In [5]:
betas = results1.betas
betas = pd.DataFrame(betas)
betas.describe()

,0,1,2,3
count,257.000000,257.000000,257.000000,257.000000
mean,145.165331,49.179859,-8.713310,25.431563
std,243.505318,101.260859,57.585723,134.092684
min,-46.967759,-102.298783,-275.959207,-298.931959
25%,5.220818,-0.901865,-6.851844,-5.890214
50%,18.309975,1.924896,0.592348,0.406515
75%,147.180948,37.015777,8.864405,30.219458
max,936.587242,546.711586,128.834048,561.420757
